### Optimization Algorithms

#### Scenario
Suppose that I have a business with three job categories of engineer, baker, and accountant. Each category has five promotion levels of 1 through 5. There are also four city locations where each of these categories of employees can work from. The company allows workers to change locations about every three years and will pay for it, but a level and position must be available. How can I use python with either scipy optimize or pytorch optimize to optimize the number of moves each year by maximizing the available moves while minimizing the total cost to staying within a certain budget?

In [1]:
import pandas as pd
import numpy as np
import itertools
from itertools import product
from scipy.optimize import milp, LinearConstraint, Bounds, linprog

In [2]:
# Categories
cities = ['Seattle', 'Los Angeles', 'Denver', 'Austin', 'Tokyo', 'London']
job_categories = ['Engineer', 'Scientist', 'Accountant']
levels = [1, 2, 3, 4, 5]

# Combining categories and adding people per category
combinations = list(itertools.product(cities, job_categories, levels))
df = pd.DataFrame(combinations, columns=['City', 'Position', 'Level'])
df['People'] = [100, 200, 200, 50, 5,
               25, 45, 55, 15, 3,
               3, 4, 5, 2, 1,
               200, 300, 350, 150, 15,
               50, 80, 95, 30, 6,
               9, 12, 9, 4, 2,
               80, 150, 150, 50, 5,
               30, 45, 55, 15, 3,
               3, 4, 5, 2, 1,
               120, 130, 140, 80, 10,
               60, 50, 50, 30, 5,
               6, 10, 9, 4, 2,
               12, 13, 14, 8, 1,
               6, 5, 5, 3, 1,
               3, 3, 3, 2, 1,
               10, 10, 10, 3, 1,
               10, 10, 10, 3, 1,
               2, 3, 2, 2, 1]

# Functions to apply additional categories 
def level_cat(level):
    '''Returns Senior if the level is 4 or above; else the level is junior'''
    if level >= 4:
        return 'Senior'
    else:
        return 'Junior'
        
def us_city(city):
    '''Returns True is the city is in the United States; else False'''
    if city in ['Seattle', 'Los Angeles', 'Denver', 'Austin']:
        return True
    else:
        return False

def cost_per_move(df):
    '''Returns the cost per move based on conditions'''
    if df.City_in_US == True and df.Level_Category == 'Junior':
        return 8000
    elif df.City_in_US == True and df.Level_Category == 'Senior':
        return 9500
    elif df.City_in_US == False and df.Level_Category == 'Junior':
        return 10000
    else:
        return 12000

# Applying functions
df['Level_Category'] = df.Level.apply(level_cat)
df['City_in_US'] = df.City.apply(us_city)
df['Cost_per_Move'] = df.apply(cost_per_move, axis=1)

# Final DataFrame
df

# Expanded rows of dataframe by number of people
expanded_df = df.reindex(df.index.repeat(df['People']))

# Reset the index if desired
expanded_df = expanded_df.reset_index(drop=True)

# Drop people
expanded_df = expanded_df.drop(columns='People')

expanded_df

,City,Position,Level,Level_Category,City_in_US,Cost_per_Move
0,Seattle,Engineer,1,Junior,True,8000
1,Seattle,Engineer,1,Junior,True,8000
2,Seattle,Engineer,1,Junior,True,8000
3,Seattle,Engineer,1,Junior,True,8000
4,Seattle,Engineer,1,Junior,True,8000
...,...,...,...,...,...,...
3482,London,Accountant,3,Junior,False,10000
3483,London,Accountant,3,Junior,False,10000
3484,London,Accountant,4,Senior,False,12000
3485,London,Accountant,4,Senior,False,12000


In [3]:
print(f'Total Number of Positions: {len(expanded_df):,}; Total Cost: ${expanded_df.Cost_per_Move.sum():,}') 

Total Number of Positions: 3,487; Total Cost: $29,001,000


In [4]:
sample_df = expanded_df.sample(frac=0.25)
print(f'Total Number of Positions: {len(sample_df):,}; Total Cost: ${sample_df.Cost_per_Move.sum():,}') 

Total Number of Positions: 872; Total Cost: $7,238,500


In [5]:
# Cost of 25% Sample size each iteration
cost_list = []
iterations = 1000
for n in range(0, iterations):
    sample_df = expanded_df.sample(frac=0.25)
    cost_list.append(sample_df.Cost_per_Move.sum()) 

cost_df = pd.DataFrame(cost_list, columns=['cost'])

print(f'Out of {iterations} iterations')
print(f'Min Cost: ${cost_df.cost.min():,}; Max Cost: ${cost_df.cost.max():,}; Avg Cost: ${cost_df.cost.mean():,.0f}; STDev Cost: ${cost_df.cost.std():,.0f}') 

Out of 1000 iterations
Min Cost: $7,183,500; Max Cost: $7,321,500; Avg Cost: $7,253,134; STDev Cost: $18,600


In [6]:
# Cost of between 20% and 30% Sample size each iteration
cost_list = []
iterations = 1000
for n in range(0, iterations):
    # Random percentage of movers
    random_mover_percentage = np.random.uniform(low=0.2, high=0.3)
    sample_df = expanded_df.sample(frac=random_mover_percentage)
    cost_list.append(sample_df.Cost_per_Move.sum()) 

cost_df = pd.DataFrame(cost_list, columns=['cost'])

print(f'Out of {iterations} iterations')
print(f'Min Cost: ${cost_df.cost.min():,}; Max Cost: ${cost_df.cost.max():,}; Avg Cost: ${cost_df.cost.mean():,.0f}; STDev Cost: ${cost_df.cost.std():,.0f}')

Out of 1000 iterations
Min Cost: $5,813,500; Max Cost: $8,732,000; Avg Cost: $7,277,550; STDev Cost: $817,990


In [7]:
cost_df.head()

,cost
0,7726000
1,6913500
2,6654500
3,8612500
4,7362500


#### Scipy

In [14]:
import pandas as pd
import itertools
from scipy.optimize import linprog

# Step 1: Define cities, jobs, levels
cities = ['Seattle', 'Los Angeles', 'Denver', 'Austin', 'Tokyo', 'London']
job_categories = ['Engineer', 'Scientist', 'Accountant']
levels = [1, 2, 3, 4, 5]
combinations = list(itertools.product(cities, job_categories, levels))

# Step 2: Create DataFrame and assign people counts
df = pd.DataFrame(combinations, columns=['City', 'Position', 'Level'])
df['People'] = [
    100, 200, 200, 50, 5,
    25, 45, 55, 15, 3,
    3, 4, 5, 2, 1,
    200, 300, 350, 150, 15,
    50, 80, 95, 30, 6,
    9, 12, 9, 4, 2,
    80, 150, 150, 50, 5,
    30, 45, 55, 15, 3,
    3, 4, 5, 2, 1,
    120, 130, 140, 80, 10,
    60, 50, 50, 30, 5,
    6, 10, 9, 4, 2,
    12, 13, 14, 8, 1,
    6, 5, 5, 3, 1,
    3, 3, 3, 2, 1,
    10, 10, 10, 3, 1,
    10, 10, 10, 3, 1,
    2, 3, 2, 2, 1
]

# Step 3: Add category flags
df['Level_Category'] = df['Level'].apply(lambda x: 'Senior' if x >= 4 else 'Junior')
df['City_in_US'] = df['City'].apply(lambda x: x in ['Seattle', 'Los Angeles', 'Denver', 'Austin'])

def cost_per_move(row):
    if row.City_in_US and row.Level_Category == 'Junior':
        return 8000
    elif row.City_in_US and row.Level_Category == 'Senior':
        return 9500
    elif not row.City_in_US and row.Level_Category == 'Junior':
        return 10000
    else:
        return 12000

df['Cost_per_Move'] = df.apply(cost_per_move, axis=1)

# Step 4: Build move options and associated costs
move_options = []
costs = []

for idx_from, row_from in df.iterrows():
    for idx_to, row_to in df.iterrows():
        if (
            row_from['City'] != row_to['City'] and
            row_from['Position'] == row_to['Position'] and
            row_from['Level'] == row_to['Level']
        ):
            max_moves = min(row_from['People'] // 4, row_to['People'] // 4)
            if max_moves > 0:
                avg_cost = (row_from['Cost_per_Move'] + row_to['Cost_per_Move']) / 2
                move_options.append((idx_from, idx_to, max_moves))
                costs.append(avg_cost)

# Step 5: Optimization setup
c = [-1] * len(move_options)  # maximize moves
A_ub = [costs]                # budget constraint
b_ub = [float('inf')]         # placeholder
bounds = [(0, max_mv) for _, _, max_mv in move_options]

# Step 6: Estimate required budget to move all eligible
total_estimated_cost = sum([max_mv * cost for (_, _, max_mv), cost in zip(move_options, costs)])
b_ub[0] = total_estimated_cost

# Step 7: Solve optimization
res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Step 8: Output result
actual_moves = int(-res.fun) if res.success else 0
actual_cost = int(sum(res.x[i] * costs[i] for i in range(len(costs)))) if res.success else 0

print("✅ Optimization successful" if res.success else "❌ Optimization failed")
print(f"Total Moves: {actual_moves}")
print(f"Total Estimated Annual Budget Required: ${actual_cost:,}")


✅ Optimization successful
Total Moves: 2028
Total Estimated Annual Budget Required: $16,829,000


In [13]:
# Step 9: Create DataFrame of selected moves
move_data = []
for i, (idx_from, idx_to, _) in enumerate(move_options):
    num_moves = res.x[i]
    if num_moves > 0.01:  # filter near-zero float noise
        from_row = df.loc[idx_from]
        to_row = df.loc[idx_to]
        move_data.append({
            'From_City': from_row['City'],
            'To_City': to_row['City'],
            'Position': from_row['Position'],
            'Level': from_row['Level'],
            'From_Level_Category': from_row['Level_Category'],
            'To_Level_Category': to_row['Level_Category'],
            'Moves': int(round(num_moves)),
            'Cost_per_Move': (from_row['Cost_per_Move'] + to_row['Cost_per_Move']) / 2,
            'Total_Cost': int(round(num_moves * ((from_row['Cost_per_Move'] + to_row['Cost_per_Move']) / 2)))
        })

moves_df = pd.DataFrame(move_data)
moves_df.head()


,From_City,To_City,Position,Level,From_Level_Category,To_Level_Category,Moves,Cost_per_Move,Total_Cost
0,Seattle,Los Angeles,Engineer,1,Junior,Junior,25,8000.0,200000
1,Seattle,Denver,Engineer,1,Junior,Junior,20,8000.0,160000
2,Seattle,Austin,Engineer,1,Junior,Junior,25,8000.0,200000
3,Seattle,Tokyo,Engineer,1,Junior,Junior,3,9000.0,27000
4,Seattle,London,Engineer,1,Junior,Junior,2,9000.0,18000


#### PuLP

In [10]:
import pandas as pd
import itertools
import pulp

# ==== Step 1: Build the DataFrame ====
cities = ['Seattle', 'Los Angeles', 'Denver', 'Austin', 'Tokyo', 'London']
job_categories = ['Engineer', 'Scientist', 'Accountant']
levels = [1, 2, 3, 4, 5]

# All combinations
combinations = list(itertools.product(cities, job_categories, levels))
df = pd.DataFrame(combinations, columns=['City', 'Position', 'Level'])

# Your provided People data
df['People'] = [
    100, 200, 200, 50, 5, 25, 45, 55, 15, 3, 3, 4, 5, 2, 1,
    200, 300, 350, 150, 15, 50, 80, 95, 30, 6, 9, 12, 9, 4, 2,
    80, 150, 150, 50, 5, 30, 45, 55, 15, 3, 3, 4, 5, 2, 1,
    120, 130, 140, 80, 10, 60, 50, 50, 30, 5, 6, 10, 9, 4, 2,
    12, 13, 14, 8, 1, 6, 5, 5, 3, 1, 3, 3, 3, 2, 1,
    10, 10, 10, 3, 1, 10, 10, 10, 3, 1, 2, 3, 2, 2, 1
]

# ==== Step 2: Helper functions ====
def level_cat(level):
    return 'Senior' if level >= 4 else 'Junior'

def us_city(city):
    return city in ['Seattle', 'Los Angeles', 'Denver', 'Austin']

def cost_per_move(row):
    if row.City_in_US and row.Level_Category == 'Junior':
        return 8000
    elif row.City_in_US and row.Level_Category == 'Senior':
        return 9500
    elif not row.City_in_US and row.Level_Category == 'Junior':
        return 10000
    else:
        return 12000

# Apply helper functions
df['Level_Category'] = df['Level'].apply(level_cat)
df['City_in_US'] = df['City'].apply(us_city)
df['Cost_per_Move'] = df.apply(cost_per_move, axis=1)

# ==== Step 3: Set parameters ====
total_budget = None  # We'll let the optimizer compute total cost from ~1/4 of population
max_fraction_move = 0.25  # At most 1/4 of a group's population moves per year

# ==== Step 4: Create move options ====
move_options = []
for i, from_row in df.iterrows():
    for j, to_row in df.iterrows():
        if i != j and from_row['Position'] == to_row['Position'] and from_row['Level'] == to_row['Level']:
            max_moves = int(from_row['People'] * max_fraction_move)
            if max_moves > 0:
                avg_cost = (from_row['Cost_per_Move'] + to_row['Cost_per_Move']) / 2
                move_options.append((i, j, max_moves, avg_cost))

# ==== Step 5: Define PuLP model ====
prob = pulp.LpProblem("Maximize_Moves", pulp.LpMaximize)

# Decision variables
move_vars = pulp.LpVariable.dicts(
    "Moves",
    ((i, j) for i, j, _, _ in move_options),
    lowBound=0,
    cat="Integer"
)

# Objective: Maximize total moves
prob += pulp.lpSum([move_vars[(i, j)] for i, j, _, _ in move_options])

# Constraints: Max moves per source group
for i in df.index:
    prob += pulp.lpSum([move_vars[(i, j)] for i2, j, _, _ in move_options if i2 == i]) <= int(df.loc[i, 'People'] * max_fraction_move)

# Constraints: Max moves per destination group
for j in df.index:
    prob += pulp.lpSum([move_vars[(i, j2)] for i, j2, _, _ in move_options if j2 == j]) <= int(df.loc[j, 'People'] * max_fraction_move)

# Optional: Budget constraint
# If we want to restrict to a budget, uncomment:
# prob += pulp.lpSum([move_vars[(i, j)] * avg_cost for i, j, _, avg_cost in move_options]) <= total_budget

# ==== Step 6: Solve ====
prob.solve(pulp.PULP_CBC_CMD(msg=0))

# ==== Step 7: Collect results ====
move_data = []
for i, j, _, avg_cost in move_options:
    num_moves = move_vars[(i, j)].value()
    if num_moves and num_moves > 0:
        from_row = df.loc[i]
        to_row = df.loc[j]
        move_data.append({
            'From_City': from_row['City'],
            'To_City': to_row['City'],
            'Position': from_row['Position'],
            'Level': from_row['Level'],
            'Moves': int(num_moves),
            'Cost_per_Move': avg_cost,
            'Total_Cost': int(num_moves * avg_cost)
        })

moves_df = pd.DataFrame(move_data)

# ==== Step 8: Calculate total budget ====
total_moves = moves_df['Moves'].sum()
total_cost = moves_df['Total_Cost'].sum()

print(f"Optimal moves per year: {total_moves}")
print(f"Total budget needed: ${total_cost:,.0f}")
moves_df.head()


Optimal moves per year: 836
Total budget needed: $6,912,000


,From_City,To_City,Position,Level,Moves,Cost_per_Move,Total_Cost
0,Seattle,Los Angeles,Engineer,1,5,8000.0,40000
1,Seattle,Denver,Engineer,1,20,8000.0,160000
2,Seattle,Los Angeles,Engineer,2,13,8000.0,104000
3,Seattle,Denver,Engineer,2,37,8000.0,296000
4,Seattle,Los Angeles,Engineer,3,50,8000.0,400000
